<a href="https://colab.research.google.com/github/gracewindheim/gmwDataSci/blob/master/q3hw8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 3

Fit a multilayer neural network to the oasis data to predict Gold_Lesions. Use 10-fold cross-validation to compare the performance where you hold out 10% of the observation and fit on the remainder, then repeat this process 10 times. Compare the cross validated error to logistic regression. Note, you won't be able to fit a very complex NN model on this data because the size is small.

In [1]:
# Load libraries
import numpy as np
import pandas as pd
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
import tensorflow as tf

# Set random seed
np.random.seed(0)

! if [ ! -e oasis.csv ];\
  then wget https://raw.githubusercontent.com/bcaffo/ds4bme_intro/master/data/oasis.csv; \
fi;

Using TensorFlow backend.


--2020-04-29 19:16:13--  https://raw.githubusercontent.com/bcaffo/ds4bme_intro/master/data/oasis.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22274 (22K) [text/plain]
Saving to: ‘oasis.csv’

oasis.csv           100%[===================>]  21.75K  --.-KB/s    in 0.01s   

2020-04-29 19:16:13 (1.76 MB/s) - ‘oasis.csv’ saved [22274/22274]



In [2]:
# Number of features
number_of_features = 12

## Read in the data and display a few rows
dataset = pd.read_csv("oasis.csv")

# Generate features matrix and target vector
features = dataset.loc[:, dataset.columns != 'GOLD_Lesions']
target = dataset['GOLD_Lesions']

features.shape

(100, 12)

## Multilayer neural network

In [0]:
# Create function returning a compiled network
def create_network():
    
    # Start neural network
    network = models.Sequential()

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=64, activation='relu', input_shape=(number_of_features,)))

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=64, activation='relu'))

    # Add fully connected layer with a sigmoid activation function
    network.add(layers.Dense(units=1, activation='sigmoid'))

    # Compile neural network
    network.compile(loss='binary_crossentropy', # Cross-entropy
                    optimizer='rmsprop', # Root Mean Square Propagation
                    metrics=['accuracy']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [0]:
# Wrap Keras model so it can be used by scikit-learn
neural_network = KerasClassifier(build_fn=create_network, 
                                 epochs=10, 
                                 batch_size=100, 
                                 verbose=0)

In [7]:
# Evaluate neural network using 10-fold cross-validation
scores = cross_val_score(neural_network, features, target, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


Accuracy: 0.71 (+/- 0.44)


## Logistic regression

In [0]:
# Create function returning a compiled logistic regression network (only inputs and outputs, no hidden layers)
def create_network():
    
    # Start neural network
    network = models.Sequential()
    network.add(layers.Dense(units=64, activation='relu', input_shape=(number_of_features,)))
    network.add(layers.Dense(units=1, activation='sigmoid'))

    # Compile neural network
    network.compile(loss='binary_crossentropy', # Cross-entropy
                    optimizer='rmsprop', # Root Mean Square Propagation
                    metrics=['accuracy']) # Accuracy performance metric
    
    # Return compiled network
    return network

In [0]:
# Wrap Keras model so it can be used by scikit-learn
neural_network2 = KerasClassifier(build_fn=create_network, 
                                 epochs=10, 
                                 batch_size=100, 
                                 verbose=0)

In [0]:
# Evaluate neural network using 10-fold cross-validation
scores2 = cross_val_score(neural_network2, features, target, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores2.mean(), scores2.std() * 2))

Accuracy: 0.60 (+/- 0.32)


## Comparison

Multilayer neural network yielded 71% accuracy while logistic regression yielded 60% accuracy.